# YamNet

This is the code use for extracting features .
Because of the package needed to run this script we extracted the Features in Google collab and we have the code here only for the mentions. You can find the extracted fetaures for train and test segments in google Drive YamNet_Features.


In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
# Load YAMNet model
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

def extract_embeddings(audio_data):
    audio_data = tf.convert_to_tensor(audio_data, dtype=tf.float32)
    scores, embeddings, spectrogram = yamnet_model(audio_data)
    return embeddings.numpy()

def load_audio(file_path, sr=16000):
    audio, original_sr = librosa.load(file_path, sr=sr)
    return audio

def save_data_with_names(base_path, save_path):
    total_files = 0

    for class_name in os.listdir(base_path):
        class_dir = os.path.join(base_path, class_name)
        if os.path.isdir(class_dir):
            for file_name in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file_name)
                if file_path.endswith('.wav'):
                    total_files += 1
                    try:
                        audio_data = load_audio(file_path)
                        embedding = extract_embeddings(audio_data)
                        if embedding.shape[0] >= 1:  # Ensure embedding is not empty
                            MEAN = np.mean(embedding, axis=0, keepdims=True)
                            
                            # Save embedding (MEAN) as X.npy
                            save_folder = os.path.join(save_path, class_name)
                            os.makedirs(save_folder, exist_ok=True)
                            np.save(os.path.join(save_folder, f'{file_name[:-4]}.npy'), MEAN)
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

    print(f"Total files found: {total_files}")
    print(f"Data saved successfully in {save_path}")


In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/data/DL_Project/UrbanSound8K/Train'
save_path = '/content/drive/MyDrive/Colab Notebooks/data/DL_Project/UrbanSound8K/YamNet_Train_Data'
load_data_from_folders(base_path, save_path)